https://adventofcode.com/2023/day/17

In [61]:
import pandas as pd

input_test = r"""2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533"""

with open('data/day_17.txt') as txtfile:
    input_day = txtfile.read()

def get_data(prod=0):
    input_data = input_day if prod else input_test
    return input_data

In [214]:
data = get_data(0)
df = pd.DataFrame([list(l) for l in data.splitlines()])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,2,4,1,3,4,3,2,3,1,1,3,2,3
1,3,2,1,5,4,5,3,5,3,5,6,2,3
2,3,2,5,5,2,4,5,6,5,4,2,5,4
3,3,4,4,6,5,8,5,8,4,5,4,5,2
4,4,5,4,6,6,5,7,8,6,7,5,3,6
5,1,4,3,8,5,9,8,7,9,8,4,5,4
6,4,4,5,7,8,7,6,9,8,7,7,6,6
7,3,6,3,7,8,7,7,9,7,9,6,5,3
8,4,6,5,4,9,6,7,9,8,6,8,8,7
9,4,5,6,4,6,7,9,9,8,6,4,5,3


In [75]:
import networkx as nx
from scipy.spatial import distance_matrix

In [215]:
history = []
shortest_k = 0, 0

In [216]:
start = shortest_k
span = len(df) - 1
sm = df.loc[start[0]:start[0]+span, start[1]:start[1]+span]
sm_u = sm.unstack().reset_index().rename(columns={0: "w"}).astype(int)
nodos = sm_u.apply(lambda x: [x["level_1"], x["level_0"]], axis=1)
sm_u["destino"] = sm_u.apply(lambda x: tuple([x["level_1"], x["level_0"]]), axis=1)
d = distance_matrix(np.array(nodos.tolist()), np.array(nodos.tolist()), p=1)
df_nx = pd.concat([nodos[d[i]==1].apply(tuple).to_frame("destino")\
    .assign(origen=lambda x: [tuple(nodos[i])]*len(x)) for i in nodos.index], ignore_index=True)\
    .merge(sm_u[["destino", "w"]], on="destino")

G = nx.from_pandas_edgelist(df_nx, "origen", "destino", ["w"])
# paths = nx.shortest_path(G, source=start, weight="w")
# path_length = nx.shortest_path_length(G, source=start, weight="w")
# shortest_k = min({k:path_length[k] for k, v in paths.items() if len(v)==span+1}, key=lambda x: path_length[x])
# shortest_path = paths[shortest_k]
# history += shortest_path
# shortest_path

In [127]:
df_copy = df.copy()
for i in history:
    df_copy.loc[i] = "#"
df_copy

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,#,#,#,3,4,3,2,3,1,1,3,2,3
1,3,2,#,5,4,5,3,5,3,5,6,2,3
2,3,2,5,5,2,4,5,6,5,4,2,5,4
3,3,4,4,6,5,8,5,8,4,5,4,5,2
4,4,5,4,6,6,5,7,8,6,7,5,3,6
5,1,4,3,8,5,9,8,7,9,8,4,5,4
6,4,4,5,7,8,7,6,9,8,7,7,6,6
7,3,6,3,7,8,7,7,9,7,9,6,5,3
8,4,6,5,4,9,6,7,9,8,6,8,8,7
9,4,5,6,4,6,7,9,9,8,6,4,5,3


In [217]:
pairs = {p[0]: p[1] for p in nx.all_pairs_dijkstra(G, weight="w")}

In [219]:
p = (0, 0)
{p: {k: pairs[p][0][k] for k, v in pairs[p][1].items() if len(v)==4}}

{(0, 0): {(1, 2): 4, (0, 3): 6, (2, 1): 6, (3, 0): 8}}